In [224]:
from train_set import *
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, classification_report, precision_recall_curve
import joblib
import sys
import os

In [225]:
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../..")))

# Model Data Set

In [226]:
def find_model_data():
        """
        A function that engineers features and creates a training dataset for the pickoff 
        likelihood model. 

        Features engineered:

        - Pitcher Handedness
        - Batter Handedness
        - Run Differential
        - Outs
        - Steal Score
        - Home Team
        - Runner Distance From Base
        
        Returns:

        Two Pandas DataFrames:
            - One DF of the features of the model
            - One Df (technically a series) of the target variable of the model
            
        """

        with db.connect("../database/smt_2025.db") as con:
            # Find all unique baserunners who have been on first base
            players_on_first = con.sql("""SELECT DISTINCT first_baserunner AS player_name FROM game_info""").df()

            # Find the number of times each distinct baserunner as been on first base
            plays_on_first = con.sql("""SELECT COUNT(*) count_on_first, first_baserunner AS player_name FROM game_info 
                                        WHERE first_baserunner != 'NA' AND second_baserunner = 'NA' 
                                            AND third_baserunner = 'NA'
                                        GROUP BY first_baserunner
                                        ORDER BY COUNT(*) DESC""").df()
            # Find all plays with a runner on first attempting to steal second.
            steal_plays = con.sql("""WITH pitches AS (SELECT * FROM 
                                (SELECT *, 
                                LEAD(event_code) OVER (PARTITION BY game_str, play_per_game ORDER BY timestamp, event_code) AS next_event,
                            LEAD(player_position) OVER (PARTITION BY game_str, play_per_game ORDER BY timestamp, event_code) AS next_player
                                FROM game_events) subquery
                                WHERE event_code = 3 AND player_position = 2 AND (next_event = 2 OR next_event = 16) AND (next_player = 4 OR next_player = 6 OR next_player = 255)
                                ORDER BY game_str, play_per_game),
                            
                            runner_info AS (SELECT * FROM 
                            game_info
                                WHERE first_baserunner != 'NA' AND second_baserunner = 'NA')
                            
                            SELECT pi.game_str, play_id, pi.play_per_game, player_position, event_code, pi.home_team, first_baserunner as player_name, second_baserunner FROM pitches pi
                                INNER JOIN runner_info ri
                                ON pi.game_str = ri.game_str AND pi.play_per_game = ri.play_per_game
                            """).df()
            
            # Find stolen base attemps for each baserunner
            steal_count = con.sql("""WITH pitches AS (SELECT * FROM 
                                (SELECT *, 
                                LEAD(event_code) OVER (PARTITION BY game_str, 
                                play_per_game ORDER BY timestamp, 
                                event_code) AS next_event,
                            LEAD(player_position) OVER (PARTITION BY game_str, 
                                play_per_game ORDER BY timestamp, 
                                event_code) AS next_player
                                FROM game_events) subquery
                                WHERE event_code = 3 
                                AND player_position = 2 
                                AND (next_event = 2 OR next_event = 16) 
                                AND (next_player = 4 OR next_player = 6 OR next_player = 255)
                                ORDER BY game_str, play_per_game),
                            
                            runner_info AS (SELECT * 
                                FROM game_info
                                WHERE first_baserunner != 'NA' AND second_baserunner = 'NA')
                            
                            SELECT COUNT(*) stolen_bases, player_name FROM
                                (SELECT pi.game_str, play_id, pi.play_per_game, player_position, event_code, pi.home_team, first_baserunner as player_name, second_baserunner FROM pitches pi
                                INNER JOIN runner_info ri
                                ON pi.game_str = ri.game_str AND pi.play_per_game = ri.play_per_game) subquery
                            GROUP BY player_name
                            ORDER BY COUNT(*) DESC""").df()
            
            # Make table for each runner who has been on first along with their steal attempts and how many times they have been on first
            df_runners = pd.merge(players_on_first, plays_on_first, on = "player_name", how = "left")
            df_thieves = pd.merge(df_runners, steal_count, on="player_name", how="left").fillna(0).sort_values(by="stolen_bases", ascending=False)

            # Calculate a "steal score" that values stolen base attempts and penalizes 
            # the more times a player has been on first base (more opportunity to steal)
            df_thieves["steal_score_initial"] = (df_thieves["stolen_bases"]/df_thieves["count_on_first"] * np.log(df_thieves["stolen_bases"])).fillna(0)
            mean_score = df_thieves[df_thieves["steal_score_initial"] != 0]["steal_score_initial"].mean()
            df_thieves["steal_score"] = df_thieves["steal_score_initial"] / mean_score * 100

            # Find pickoff plays with a runner on first base
            df_pickoff_plays = con.sql("""
                                WITH first_base_player_pos AS (SELECT game_str, play_id, timestamp, player_position AS runner, field_x, field_y
                                    FROM(
                                        SELECT *, DENSE_RANK() OVER (PARTITION BY game_str, play_id ORDER BY timestamp) as time_order
                                        FROM player_pos 
                                        WHERE player_position = 11
                                        ORDER BY game_str, play_id, timestamp) sub1
                                        WHERE time_order = 1)
                                SELECT ge.game_str, ge.play_id, player_position, event_code, ge.home_team, top_bottom_inning, pitcher, batter, first_baserunner as player_name, field_x, field_y 
                                FROM game_events ge
                                LEFT JOIN game_info gi 
                                ON ge.game_str = gi.game_str AND ge.play_per_game = gi.play_per_game
                                INNER JOIN first_base_player_pos fb
                                ON ge.game_str = fb.game_str AND ge.play_id = fb.play_id
                                WHERE player_position = 1 AND event_code = 6 
                                    AND second_baserunner = 'NA' AND third_baserunner = 'NA' """).df()
            # Find plays that are pitches not pickoffs
            df_pitches = con.sql("""
                                WITH first_base_player_pos AS (SELECT game_str, play_id, timestamp, player_position AS runner, field_x, field_y
                                    FROM(
                                        SELECT *, DENSE_RANK() OVER (PARTITION BY game_str, play_id ORDER BY timestamp) as time_order
                                        FROM player_pos 
                                        WHERE player_position = 11
                                        ORDER BY game_str, play_id, timestamp) sub1
                                        WHERE time_order = 1)
                                SELECT ge.game_str, ge.play_id, player_position, event_code, ge.home_team, top_bottom_inning, pitcher, batter, first_baserunner as player_name, field_x, field_y 
                                FROM game_events ge
                                LEFT JOIN game_info gi 
                                ON ge.game_str = gi.game_str AND ge.play_per_game = gi.play_per_game
                                INNER JOIN first_base_player_pos fb
                                ON ge.game_str = fb.game_str AND ge.play_id = fb.play_id
                                WHERE player_position = 1 AND event_code = 1 
                                    AND second_baserunner = 'NA' AND third_baserunner = 'NA' 
                                """).df()
            # Signify pickoff vs. not pickoff
            df_pickoff_plays["pickoff"] = 1
            df_pitches["pickoff"] = 0

            # # Have the split of data where there are 'ratio' times as many regular pitches as pickoffs
            # df_pitches_model = df_pitches.sample(n=ratio*df_pickoff_plays.shape[0])

            # Concatenate pitch and pickoffs plays into one model dataset
            df_model_data = pd.concat((df_pickoff_plays, df_pitches), axis=0, join='inner', ignore_index=True, keys=None)

            # Find the distance of the runner from the front right corner of first base (3 seconds before pitch thrown)
            df_model_data["lead_distance"] = np.sqrt(pow(df_model_data["field_x"]-63.63, 2) + pow(df_model_data["field_y"]-63.63, 2)) - 1.25
            # Merge to add steal_score feature
            df_model_data = pd.merge(df_model_data, df_thieves, on="player_name", how = "left")
            # Binary is_home feature
            df_model_data["is_home"] = np.where(df_model_data['top_bottom_inning'] == "top", 1, 0)

            # Pitcher handedness: using the side of the y-axis to determine where the pitcher releases the ball
            df_pitcher_hand = con.sql("""WITH rp AS 
                            (SELECT * FROM
                            (SELECT ball_position_x, play_id, game_str,
                            DENSE_RANK() OVER (PARTITION BY game_str, play_id ORDER BY timestamp) AS rank
                            FROM ball_pos bp) AS subquery
                            WHERE rank = 1), 

                            pitcher_rp AS (
                            SELECT rp.game_str, rp.ball_position_x, pitcher 
                            FROM rp
                            LEFT JOIN game_info gi
                            ON rp.game_str = gi.game_str AND rp.play_id = gi.play_per_game)
                            
                            SELECT AVG(ball_position_x) avg_rel_point, pitcher FROM pitcher_rp
                            GROUP BY pitcher;""").df()
            # Categorize lefties with a '1' and righties with a '0'
            df_pitcher_hand["pitcher_hand"] = np.where(df_pitcher_hand["avg_rel_point"] > 0, 1, 0)
            df_model_data = pd.merge(df_model_data, df_pitcher_hand, on="pitcher", how="left")

            # Batter handedness: using the side of the y-axis to determine what box the hitter stands in
            df_batter_hand = con.sql("""WITH bh AS 
                            (SELECT * FROM
                            (SELECT field_x, play_id, game_str,
                            DENSE_RANK() OVER (PARTITION BY game_str, play_id ORDER BY timestamp) AS rank
                            FROM player_pos pp
                            WHERE player_position = 10) AS subquery
                            WHERE rank = 1),
                                
                            batter_rp AS (
                            SELECT bh.game_str, bh.field_x, batter 
                            FROM bh
                            LEFT JOIN game_info gi
                            ON bh.game_str = gi.game_str AND bh.play_id = gi.play_per_game)
                                
                            SELECT AVG(field_x) avg_stance, batter FROM batter_rp
                            GROUP BY batter""").df()
            
            # Categorize lefties with a '1' and righties with a '0'
            df_batter_hand["batter_hand"] = np.where(df_batter_hand["avg_stance"] > 0, 1, 0)
            df_model_data = pd.merge(df_model_data, df_batter_hand, on="batter", how="left")

            # Find the runs scored on each play and then calculate the current score/run differential of each game
            df_runs_per_play = con.sql(""" 
                        WITH run_scored AS (SELECT play_id, game_str, SUM(run) As runs FROM
                        (SELECT DISTINCT play_id, game_str, 
                        CASE 
                            WHEN (player_position = 11 AND abs(field_x) < 2 AND abs(field_y) < 2) THEN 1 
                            ELSE 0
                        END AS run          
                        FROM player_pos
                        UNION ALL
                        SELECT DISTINCT play_id, game_str, 
                        CASE 
                            WHEN (player_position = 12 AND abs(field_x) < 2 AND abs(field_y) < 2) THEN 1 
                            ELSE 0
                        END AS run
                        FROM player_pos
                        UNION ALL
                        SELECT DISTINCT play_id, game_str, 
                        CASE 
                            WHEN (player_position = 13 AND abs(field_x) < 2 AND abs(field_y) < 2) THEN 1 
                            ELSE 0
                        END AS run
                        FROM player_pos) subquery
                        GROUP BY game_str, play_id),
                                
                        run_plays AS (SELECT gi.game_str, runs, top_bottom_inning,
                        LEAD(play_per_game) OVER (PARTITION BY gi.game_str, top_bottom_inning ORDER BY play_per_game) AS play_per_game
                        FROM game_info gi
                        LEFT JOIN run_scored AS rs
                        ON rs.play_id = gi.play_per_game AND rs.game_str = gi.game_str)
                                
                        SELECT game_str, play_per_game, runs,
                            SUM(CASE WHEN top_bottom_inning = 'top' THEN runs ELSE 0 END)
                            OVER (PARTITION BY game_str ORDER BY play_per_game
                                ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS away_score,
                            SUM(CASE WHEN top_bottom_inning = 'bottom' THEN runs ELSE 0 END)
                                OVER (PARTITION BY game_str ORDER BY play_per_game
                                ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS home_score
                        FROM run_plays
                        ORDER BY game_str, play_per_game;""").df()
        
            df_runs_per_play["run_diff"] = df_runs_per_play["home_score"] - df_runs_per_play["away_score"]
            df_model_data = pd.merge(df_model_data, df_runs_per_play, left_on=['game_str', 'play_id'], right_on=['game_str', 'play_per_game'], how='left')

            # Find the number of outs recorded on a play --> how many outs there are in the inning on the next play 
            df_outs = con.sql("""WITH runner_info AS
                                    (SELECT *, 
                                    LAG(first_baserunner) OVER (PARTITION BY game_str, top_bottom_inning ORDER BY play_per_game) AS next_first,
                                    LAG(second_baserunner) OVER (PARTITION BY game_str, top_bottom_inning ORDER BY play_per_game) AS next_second,
                                    LAG(third_baserunner) OVER (PARTITION BY game_str, top_bottom_inning ORDER BY play_per_game) AS next_third
                                    FROM game_info)

                                    SELECT DISTINCT game_str, play_per_game, SUM(out) as outs
                                    FROM
                                    (SELECT game_str, top_bottom_inning, play_per_game, 
                                    CASE 
                                        WHEN first_baserunner NOT IN (next_first, next_second, next_third) THEN 1 
                                        ELSE 0
                                        END AS out
                                    FROM runner_info
                                    UNION ALL
                                    SELECT game_str, top_bottom_inning, play_per_game, 
                                    CASE 
                                        WHEN second_baserunner NOT IN (next_first, next_second, next_third) THEN 1 
                                        ELSE 0
                                        END AS out
                                    FROM runner_info
                                    UNION ALL
                                    SELECT game_str, top_bottom_inning, play_per_game, 
                                    CASE 
                                        WHEN third_baserunner NOT IN (next_first, next_second, next_third) THEN 1 
                                        ELSE 0
                                        END AS out
                                    FROM runner_info) subquery
                                    GROUP BY game_str, play_per_game
                                    """).df()
            
            df_outs["outs"] = np.where(df_outs["outs"] > 3, 0, df_outs["outs"])
            df_model_data = pd.merge(df_model_data, df_outs, left_on=['game_str', 'play_id'], right_on=['game_str', 'play_per_game'], how='left')

            # Remove rows with any NA values
            df_model_data = df_model_data.dropna()
            df_model_data.to_csv("model_dataset.csv")

            X = df_model_data[['outs','run_diff', 'pitcher_hand', 
                                    'batter_hand', 'lead_distance', 'steal_score', 'is_home'
                                    ]]
            y = df_model_data[["pickoff"]]

            return X, y

# Train, Validation, Test Split

In [227]:
def split_model_data(val_prop: int, test_prop: int):
            # Access features and target variables from model dataset
            X, y = find_model_data()

            # First split into train/validation and test set
            X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=test_prop, stratify=y, random_state=1313)

            # Calculate the proportion of validation set rows from the new train+val set
            new_val_prop = val_prop / (1 - test_prop)

            # Split into separate train and val sets
            X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=new_val_prop, stratify=y_train_val, random_state=42)

            # Return split data in dictionary format
            return {"X": [X_train, X_val, X_test],
                    "y": [y_train, y_val, y_test]} 

In [228]:
# Generate split_data to use for all the models
split_data = split_model_data(val_prop=0.15, test_prop=0.15)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


# Modeling

In [229]:
# Model Results

model_results = pd.DataFrame()

## Logistic Regression Modeling

In [230]:
from sklearn.metrics import confusion_matrix
def logistic_regression_model(split_data):
        # Put the splits into accessible vars
        X_train, X_val, X_test = split_data["X"]
        y_train, y_val, y_test = split_data["y"]

        # Initialize model pipeline with scaling and model type (Logit)
        pipeline = Pipeline([
                ("scaler", StandardScaler()),
                ("logit", LogisticRegression(max_iter=1000))
        ])

        # Set parameters for Grid Search
        param_grid = {
                "logit__C": [0.01, 0.1, 1, 10, 100],
                "logit__solver": ["liblinear", "saga"],
                "logit__penalty": ["l1", "l2"]
        }

        # Initialize grid using f1 score and 5 folds
        grid = GridSearchCV(
            estimator=pipeline,
            param_grid=param_grid,
            scoring="f1",
            cv=5,
            n_jobs=-1,
            verbose=0
        )

        # Fit the gird and select the best model
        grid.fit(X_train, y_train)
        best_model = grid.best_estimator_

        # Probability of pickoffs on validation set
        val_probs = best_model.predict_proba(X_val)[:, 1]

        # Calculate raw f1_score from prec. and rec.
        precisions, recalls, thresholds = precision_recall_curve(y_val, val_probs)
        f1_scores = 2 * (precisions * recalls) / (precisions + recalls + 1e-10)

        # Select threshold with highest f1 score
        best_idx = f1_scores.argmax()
        best_threshold = thresholds[best_idx]

        # Probability of pickoffs on validation set
        test_probs = best_model.predict_proba(X_test)[:, 1]
        test_preds = (test_probs >= best_threshold).astype(int)

        test_acc = accuracy_score(y_test, test_preds)
        test_precision = precision_score(y_test, test_preds)
        test_recall = recall_score(y_test, test_preds)
        test_f1 = f1_score(y_test, test_preds)

        df_test_metrics = pd.DataFrame({'model_type': ["logistic_regression"], 
                                        'accuracy': [round(test_acc, 3)],
                                        'precision': [round(test_precision, 3)],
                                        'recall': [round(test_recall, 3)],
                                        'f1': [round(test_f1, 3)]})

        confusion_mat = confusion_matrix(y_test, test_preds, labels=None, sample_weight=None, normalize=None)


        return best_model, df_test_metrics, confusion_mat

In [231]:
logit_model, logit_test_metrics, logit_confusion_mat = logistic_regression_model(split_data)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/utils/validation.py:1406: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/utils/validation.py:1406: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/utils/validation.py:1406: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/uti

In [232]:
logit_confusion_mat

array([[ 934, 1174],
       [  51,  129]])

In [233]:
columns = ['outs', 'run_diff', 'pitcher_hand', 'batter_hand', 'lead_distance', 'steal_score', 'is_home']

X = pd.DataFrame([[0, 4, 1, 1, 11, 37, 1]], columns=columns)

pred = logit_model.predict_proba(X)[:, 1]

In [234]:
float(round(pred[0],4))

0.0946

# Random Forest Modeling

In [235]:
def random_forest_model(split_data):
        X_train, X_val, X_test = split_data["X"]
        y_train, y_val, y_test = split_data["y"]

        rf_model = RandomForestClassifier(n_estimators=100, class_weight="balanced", random_state=42)
        rf_model.fit(X_train, y_train)

        val_probs = rf_model.predict_proba(X_val)[:, 1]

        precisions, recalls, thresholds = precision_recall_curve(y_val, val_probs)
        f1_scores = 2 * (precisions * recalls) / (precisions + recalls + 1e-10)
        best_idx = f1_scores.argmax()
        best_threshold = thresholds[best_idx]

        print(f"Best threshold (validation): {best_threshold:.3f}")
        
        val_preds = (val_probs >= best_threshold).astype(int)

        val_auc = roc_auc_score(y_val, val_probs)
        val_acc = accuracy_score(y_val, val_preds)
        val_precision = precision_score(y_val, val_preds)
        val_recall = recall_score(y_val, val_preds)
        val_f1 = f1_score(y_val, val_preds)

        print("\n[Validation Metrics]")
        print(f"AUC:       {val_auc:.3f}")
        print(f"Accuracy:  {val_acc:.3f}")
        print(f"Precision: {val_precision:.3f}")
        print(f"Recall:    {val_recall:.3f}")
        print(f"F1 Score:  {val_f1:.3f}")

        test_probs = rf_model.predict_proba(X_test)[:, 1]
        test_preds = (test_probs >= best_threshold).astype(int)

        test_acc = accuracy_score(y_test, test_preds)
        test_precision = precision_score(y_test, test_preds)
        test_recall = recall_score(y_test, test_preds)
        test_f1 = f1_score(y_test, test_preds)

        df_test_metrics = pd.DataFrame({'model_type': ["random_forest"], 
                                        'accuracy': [round(test_acc, 3)],
                                        'precision': [round(test_precision, 3)],
                                        'recall': [round(test_recall, 3)],
                                        'f1': [round(test_f1, 3)]})


        confusion_mat = confusion_matrix(y_test, test_preds, labels=None, sample_weight=None, normalize=None)

        return rf_model, df_test_metrics, confusion_mat

In [236]:
rf_model, rf_test_metrics, rf_confusion_mat = random_forest_model(split_data)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/base.py:1365: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Best threshold (validation): 0.100

[Validation Metrics]
AUC:       0.623
Accuracy:  0.773
Precision: 0.146
Recall:    0.389
F1 Score:  0.212


In [247]:
rf_confusion_mat

array([[1712,  396],
       [ 111,   69]])

In [237]:
joblib.dump(rf_model, 'rf_model.pkl')

['rf_model.pkl']

In [ ]:
def knn_model(split_data, n_neighbors=5):
        X_train, X_val, X_test = split_data["X"]
        y_train, y_val, y_test = split_data["y"]

        pipeline = Pipeline([
            ("scaler", StandardScaler()),
            ("knn", KNeighborsClassifier(n_neighbors=n_neighbors))
        ])

        pipeline.fit(X_train, y_train)

        val_probs = pipeline.predict_proba(X_val)[:, 1]

        precisions, recalls, thresholds = precision_recall_curve(y_val, val_probs)
        f1_scores = 2 * (precisions * recalls) / (precisions + recalls + 1e-10)
        best_idx = f1_scores.argmax()
        best_threshold = thresholds[best_idx]

        print(f"Best threshold (validation): {best_threshold:.3f}")

        val_preds = (val_probs >= best_threshold).astype(int)

        val_auc = roc_auc_score(y_val, val_probs)
        val_acc = accuracy_score(y_val, val_preds)
        val_precision = precision_score(y_val, val_preds)
        val_recall = recall_score(y_val, val_preds)
        val_f1 = f1_score(y_val, val_preds)

        print("\n[Validation Metrics]")
        print(f"AUC:       {val_auc:.3f}")
        print(f"Accuracy:  {val_acc:.3f}")
        print(f"Precision: {val_precision:.3f}")
        print(f"Recall:    {val_recall:.3f}")
        print(f"F1 Score:  {val_f1:.3f}")

        test_probs = pipeline.predict_proba(X_test)[:, 1]
        test_preds = (test_probs >= best_threshold).astype(int)

        test_acc = accuracy_score(y_test, test_preds)
        test_precision = precision_score(y_test, test_preds)
        test_recall = recall_score(y_test, test_preds)
        test_f1 = f1_score(y_test, test_preds)

        df_test_metrics = pd.DataFrame({'model_type': ["knn"], 
                                        'accuracy': [round(test_acc, 3)],
                                        'precision': [round(test_precision, 3)],
                                        'recall': [round(test_recall, 3)],
                                        'f1': [round(test_f1, 3)]})

        confusion_mat = confusion_matrix(y_test, test_preds, labels=None, sample_weight=None, normalize=None)

        return pipeline, df_test_metrics, confusion_mat

In [239]:
knn_pipeline, knn_test_metrics, knn_confusion_mat = knn_model(split_data)

Best threshold (validation): 0.200

[Validation Metrics]
AUC:       0.609
Accuracy:  0.692
Precision: 0.124
Recall:    0.483
F1 Score:  0.198


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/neighbors/_classification.py:239: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


In [240]:
knn_confusion_mat

array([[1488,  620],
       [  85,   95]])

In [241]:
def xg_boost_model(split_data):
        X_train, X_val, X_test = split_data["X"]
        y_train, y_val, y_test = split_data["y"]

        xgb_model = XGBClassifier(
            use_label_encoder=False,
            eval_metric='logloss',
            scale_pos_weight=float((y_train == 0).sum() / (y_train == 1).sum()), 
            random_state=42
        )
        xgb_model.fit(X_train, y_train)

        val_probs = xgb_model.predict_proba(X_val)[:, 1]

        precisions, recalls, thresholds = precision_recall_curve(y_val, val_probs)
        f1_scores = 2 * (precisions * recalls) / (precisions + recalls + 1e-10)
        best_idx = f1_scores.argmax()
        best_threshold = thresholds[best_idx]

        print(f"Best threshold (validation): {best_threshold:.3f}")
        
        val_preds = (val_probs >= best_threshold).astype(int)

        val_auc = roc_auc_score(y_val, val_probs)
        val_acc = accuracy_score(y_val, val_preds)
        val_precision = precision_score(y_val, val_preds)
        val_recall = recall_score(y_val, val_preds)
        val_f1 = f1_score(y_val, val_preds)

        print("\n[Validation Metrics]")
        print(f"AUC:       {val_auc:.3f}")
        print(f"Accuracy:  {val_acc:.3f}")
        print(f"Precision: {val_precision:.3f}")
        print(f"Recall:    {val_recall:.3f}")
        print(f"F1 Score:  {val_f1:.3f}")

        test_probs = xgb_model.predict_proba(X_test)[:, 1]
        test_preds = (test_probs >= best_threshold).astype(int)

        test_acc = accuracy_score(y_test, test_preds)
        test_precision = precision_score(y_test, test_preds)
        test_recall = recall_score(y_test, test_preds)
        test_f1 = f1_score(y_test, test_preds)

        df_test_metrics = pd.DataFrame({'model_type': ["random_forest"], 
                                        'accuracy': [round(test_acc, 3)],
                                        'precision': [round(test_precision, 3)],
                                        'recall': [round(test_recall, 3)],
                                        'f1': [round(test_f1, 3)]})

        confusion_mat = confusion_matrix(y_test, test_preds, labels=None, sample_weight=None, normalize=None)

        return xgb_model, df_test_metrics, confusion_mat

In [242]:
xgb_model, xg_test_metrics, xg_confusion_mat = xg_boost_model(split_data)

/var/folders/wn/1h4z_5y959g0lxlcphs4q6bc0000gn/T/ipykernel_37712/3190200589.py:8: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  scale_pos_weight=float((y_train == 0).sum() / (y_train == 1).sum()),
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [20:57:58] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Best threshold (validation): 0.327

[Validation Metrics]
AUC:       0.689
Accuracy:  0.601
Precision: 0.127
Recall:    0.694
F1 Score:  0.215


In [243]:
xg_confusion_mat

array([[1245,  863],
       [  68,  112]])

In [246]:
pd.concat((logit_test_metrics, rf_test_metrics, knn_test_metrics, xg_test_metrics)).to_csv("model_results.csv")

In [245]:
logit_test_metrics

,model_type,accuracy,precision,recall,f1
0,logistic_regression,0.465,0.099,0.717,0.174
